In [1]:
import numpy as np
import math
import cv2
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [2]:
### Read the csv
import os
import csv

samples = []
csv_path = './data/driving_log.csv'
with open(csv_path) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
samples = samples[1:] # gets rid of the header

In [3]:
### Preprocess the image
def preprocess_image(image_data):
    image = image_data[50:140, :] # Crop image[y: y + h, x: x + w]
    image = cv2.resize(image,(66,200)) # resizing the image x,y
    return image

In [4]:
### Image Augmentation

# Flips about the vertical axis
def flip_image(image_data):
    return cv2.flip(image_data, 1)

# Converts left and right images to center images  
def multiple_cameras(path, line):
    name_left = path + '/IMG/'+ line[1].split('/')[-1]
    left_image = mpimg.imread(name_left)
    left_image = preprocess_image(left_image)
    name_right = path + '/IMG/'+ line[2].split('/')[-1]
    right_image = mpimg.imread(name_right)
    right_image = preprocess_image(right_image)
    center_angle = float(line[3])
    left_angle = center_angle + 0.25
    right_angle = center_angle - 0.25
    return left_image, right_image, left_angle, right_angle

# Randomly changes brightness of the images
def brightness(image_data):
    dst = cv2.cvtColor(image_data, cv2.COLOR_RGB2HSV)
    dst[:,:,2] = dst[:,:,2]*np.random.uniform(low = 0.4, high = 1.0)
    return cv2.cvtColor(dst, cv2.COLOR_HSV2RGB)

# Randomly adds a rectangular block of shadow to the images
def shadow(image_data):
    dst = cv2.cvtColor(image_data, cv2.COLOR_RGB2HLS)
    col = np.random.randint(low=0,high=30)
    row = np.random.randint(low=100,high=140)
    dst[row:row+60,col:col+30,1] = dst[row:row+60,col:col+30,1]*0.7
    return cv2.cvtColor(dst, cv2.COLOR_HLS2RGB)  

# Shifts the images horizontally and vertically
def shift_horizontal(image_data, center_angle):
    tx = np.random.randint(low = -30, high = 30)
    ty = np.random.randint(low = -10, high = 10)
    M = np.float32([[1,0, tx],[0,1,ty]])
    steer = np.sign(tx)*0.009 + center_angle
    return cv2.warpAffine(image_data,M,(66,200)), steer

In [5]:
### Python Generator
def generator(samples):
    num_samples = len(samples)
    BATCH_SIZE = 128
    path = './data'
    while 1:   
        shuffle(samples)
        for offset in range(0, num_samples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            image_list = []
            angle = []
            batch_samples = samples[offset:end]
            for line in batch_samples:
                name = path + '/IMG/'+ line[0].split('/')[-1]
                center_image = mpimg.imread(name)
                center_image = preprocess_image(center_image)
                image_list.append(center_image)
                center_angle = float(line[3])
                angle.append(center_angle)
                if center_angle != 0:
                    center_image_flip = flip_image(center_image)
                    image_list.append(center_image_flip)
                    angle.append(-center_angle)
                    center_image_bright = brightness(center_image)
                    center_image_shadow = shadow(center_image)
                    image_list.extend([center_image_bright, center_image_shadow])
                    angle.extend([center_angle, center_angle])    
                image_shift_horizontal, angle_horizontal_shift = shift_horizontal(center_image, center_angle)
                left_image, right_image, left_angle, right_angle = multiple_cameras(path, line)
                image_list.extend([left_image, right_image, image_shift_horizontal])
                angle.extend([left_angle, right_angle, angle_horizontal_shift])
            X = np.array(image_list)
            y = np.array(angle)
            yield shuffle(X, y)

In [6]:
### Model architecture: NVDIA
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers import Cropping2D
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Lambda(lambda x: (x / 255) - 0.5, output_shape = (200,66,3), input_shape = (200,66,3))) 
model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(2,2)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Dense(50))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Dense(10))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(p = 0.5))
model.add(Dense(1))

Using TensorFlow backend.


In [ ]:
### Configures the learning process
model.compile(optimizer='adam',loss='mean_squared_error')

### Split into train samples and validation samples
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

### Train the model
train_generator = generator(train_samples) 
validation_generator = generator(validation_samples)
        
history = model.fit_generator(train_generator,samples_per_epoch=55000, 
        validation_data=validation_generator, nb_val_samples=len(validation_samples),nb_epoch=1)

### Save the model
model.save('./model.h5')  # creates a HDF5 file

Epoch 1/1
51583/55000 [===========================>..] - ETA: 36s - loss: 0.0438